<a href="https://colab.research.google.com/github/Varora-24/Machine-Learning-Model-to-add-Salt-and-Peppper-Noise-and-then-Denoise-/blob/main/saltandpepper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('Lennadidi.png')

def add_salt_and_pepper_noise(img, amount=0.05, salt_vs_pepper=0.5):
    noisy = img.copy()
    total_pixels = img.shape[0] * img.shape[1]
    num_salt = np.ceil(amount * total_pixels * salt_vs_pepper)
    num_pepper = np.ceil(amount * total_pixels * (1.0 - salt_vs_pepper))

    # Add salt
    coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in img.shape[:2]]
    noisy[coords[0], coords[1]] = 255

    # Add pepper
    coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in img.shape[:2]]
    noisy[coords[0], coords[1]] = 0

    return noisy

# Apply salt and pepper noise
noisy_image = add_salt_and_pepper_noise(image, amount=0.05)

# Save or display the result
cv2.imwrite('Lennadidi_noisy.jpg', noisy_image)
# cv2.imshow('Salt and Pepper Noise', noisy_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


True

In [3]:
import cv2

# Load the noisy image
img = cv2.imread('Lennadidi_noisy.jpg')

# Apply median filter with a 3x3 kernel
denoised_img = cv2.medianBlur(img, 3)

# Save or display the denoised image
cv2.imwrite('Lennadidi_denoised.jpg', denoised_img)
# cv2.imshow('Denoised Image', denoised_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


True

In [4]:
import cv2
import numpy as np

# Load images
original = cv2.imread('Lennadidi.png')
noisy = cv2.imread('Lennadidi_noisy.jpg')
denoised = cv2.imread('Lennadidi_denoised.jpg')

# Check if images were loaded successfully
if original is None:
    print("Error: Could not load original image. Please check the file path 'Lennadidi.jpg'.")
elif noisy is None:
    print("Error: Could not load noisy image. Please check the file path 'Lennadidi_noisy.jpg'.")
elif denoised is None:
    print("Error: Could not load denoised image. Please check the file path 'Lennadidi_denoised.jpg'.")
else:
    # Optionally resize images to same height (if needed)
    height = min(original.shape[0], noisy.shape[0], denoised.shape[0])
    width = min(original.shape[1], noisy.shape[1], denoised.shape[1])

    original = cv2.resize(original, (width, height))
    noisy = cv2.resize(noisy, (width, height))
    denoised = cv2.resize(denoised, (width, height))

    # Concatenate images horizontally
    combined = np.hstack((original, noisy, denoised))

    # Save or display the combined image
    cv2.imwrite('comparison_frame.jpg', combined)
    # cv2.imshow('Comparison', combined)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [5]:
import cv2
import numpy as np
from math import log10, sqrt

# Function to safely load an image with error handling
def load_image(path):
    img = cv2.imread(path)
    if img is None:
        raise FileNotFoundError(f"Image not found or unable to load: {path}")
    return img

# Function to add salt and pepper noise
def add_salt_and_pepper_noise(img, amount=0.05, salt_vs_pepper=0.5):
    noisy = img.copy()
    total_pixels = img.shape[0] * img.shape[1]
    num_salt = np.ceil(amount * total_pixels * salt_vs_pepper)
    num_pepper = np.ceil(amount * total_pixels * (1.0 - salt_vs_pepper))

    # Add salt noise (white pixels)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in img.shape[:2]]
    noisy[coords[0], coords[1]] = 255

    # Add pepper noise (black pixels)
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in img.shape[:2]]
    noisy[coords[0], coords[1]] = 0

    return noisy

# Function to calculate PSNR
def calculate_psnr(original, compared):
    mse = np.mean((original - compared) ** 2)
    if mse == 0:
        return float('inf')  # No difference
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

# Load the original image
original_path = 'Lennadidi.png'
original = load_image(original_path)

# Add salt and pepper noise
noisy = add_salt_and_pepper_noise(original, amount=0.05)
cv2.imwrite('Lennadidi_noisy.jpg', noisy)

# Apply median filter to remove noise
denoised = cv2.medianBlur(noisy, 3)
cv2.imwrite('Lennadidi_denoised.jpg', denoised)

# Resize images to the smallest common size for concatenation
height = min(original.shape[0], noisy.shape[0], denoised.shape[0])
width = min(original.shape[1], noisy.shape[1], denoised.shape[1])

original_resized = cv2.resize(original, (width, height))
noisy_resized = cv2.resize(noisy, (width, height))
denoised_resized = cv2.resize(denoised, (width, height))

# Concatenate images horizontally for side-by-side comparison
combined = np.hstack((original_resized, noisy_resized, denoised_resized))
cv2.imwrite('comparison_frame.jpg', combined)

# Calculate PSNR values
psnr_noisy = calculate_psnr(original_resized, noisy_resized)
psnr_denoised = calculate_psnr(original_resized, denoised_resized)

# Calculate the improvement percentage of denoising over noisy image
if psnr_noisy == 0:
    improvement = float('inf')  # Avoid division by zero
else:
    improvement = ((psnr_denoised - psnr_noisy) / psnr_noisy) * 100

# Print PSNR and improvement results
print(f"PSNR between original and noisy image: {psnr_noisy:.2f} dB")
print(f"PSNR between original and denoised image: {psnr_denoised:.2f} dB")
print(f"Denoising improved the PSNR by: {improvement:.2f}%")


PSNR between original and noisy image: 40.96 dB
PSNR between original and denoised image: 35.48 dB
Denoising improved the PSNR by: -13.38%
